In [ ]:
from langchain.document_loaders import PyPDFLoader
file_path = 'G:\\Python coding\\Metropolia\\ML Projects\\Dataset\\autobiography-of-a-yogi.pdf'
loader=PyPDFLoader(file_path)
docs=loader.load()
docs

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=splitter.split_documents(docs)
documents[:5] # it give top 5 document chunks

In [ ]:
#vector embeddings and create vector store

from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
db=FAISS.from_documents(docs[:5],OllamaEmbeddings())

#query="who is author of Autobiography of a Yogi"
#result=db.similarity_serach(query)
#result[0].page_content

In [ ]:
from langchain_community.llms import Ollama
llm=Ollama(model="llama2")

In [ ]:
# Prompt
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import RetrievalQA
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(
        input_variables=["context", "question"],
        template=template,
    )

llm = Ollama(model="llama3.2", callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))
qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=db.as_retriever(),
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
    )

result = qa_chain({"query": "Who is author of Autobiography of an yogi"})

In [ ]:
print(result)